In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 8.4MB/s ta 0:00:011
Libraries imported.


In [3]:
CLIENT_ID = 'U4NC34IBIHVT4XWCPSFXPJ0SWTPRXLDZEKPHLJJRJIHLNDA3' # your Foursquare ID
CLIENT_SECRET = 'JF1OUBHMF1FWH25QSZWI1P14CYHAZCLCGH41GZZFVPODFVWH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U4NC34IBIHVT4XWCPSFXPJ0SWTPRXLDZEKPHLJJRJIHLNDA3
CLIENT_SECRET:JF1OUBHMF1FWH25QSZWI1P14CYHAZCLCGH41GZZFVPODFVWH


In [4]:
cities = ["Seattle, WA", 'Chicago, IL', 'San Francisco, CA', 'New York, NY', 'Oahu, HI']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d181941735") # Museum CATEGORY ID
    results[city] = requests.get(url).json()

In [5]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of  Museum in {city} = ", results[city]['response']['totalResults'])


Total number of  Museum in Seattle, WA =  27
Total number of  Museum in Chicago, IL =  63
Total number of  Museum in San Francisco, CA =  49
Total number of  Museum in New York, NY =  119
Total number of  Museum in Oahu, HI =  58


In [19]:

maps[cities[0]]

In [8]:

maps[cities[1]]

In [9]:

maps[cities[2]]

In [10]:

maps[cities[3]]

In [11]:

maps[cities[4]]

In [17]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean center")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Seattle, WA
Mean Distance from Mean center
0.0354866867517917
Chicago, IL
Mean Distance from Mean center
0.05911550326905858
San Francisco, CA
Mean Distance from Mean center
0.036803481731972135
New York, NY
Mean Distance from Mean center
0.02866588539627982
Oahu, HI
Mean Distance from Mean center
0.05654271267288334


In [13]:

maps[cities[0]]

In [14]:

maps[cities[1]]

In [15]:

maps[cities[2]]

In [16]:

maps[cities[3]]

In [18]:
maps[cities[4]]